In [1]:
#written by Noah Friedman (a template for scripts to be excuted in the spyder environment
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', 275000)

In [5]:
def get_per_case_oncogenic_mut_info(muts):
    oncogenicMuts = muts[muts['oncogenic'].notnull()]
    nMutOncDict = dict(oncogenicMuts['Tumor_Sample_Barcode'].value_counts())
    return nMutOncDict

#TODO flesh this out
def get_per_case_unique_gene_oncogenic_mut_info(muts):
    oncogenicMuts = muts[muts['oncogenic'].notnull()]
    oncogenicMuts['caseGeneAltered'] = oncogenicMuts.apply(lambda row: row['Tumor_Sample_Barcode'] + '_' + row['Hugo_Symbol'], axis=1)
    oncogenicMuts = oncogenicMuts.drop_duplicates(subset=['caseGeneAltered'])
    return dict(oncogenicMuts['Tumor_Sample_Barcode'].value_counts())

In [4]:
def get_per_case_mut_info(nmutDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/nmutInfo_impact_filtered.tsv'):
    df = pd.read_table(nmutDfPath)
    return dict(zip(df['Tumor_Sample_Barcode'], df['Nmut']))

def get_per_case_unique_gene_mut_info(muts):
    muts['caseGeneAltered'] = muts.apply(lambda row: row['Tumor_Sample_Barcode'] + '_' + row['Hugo_Symbol'], axis=1)
    muts = muts.drop_duplicates(subset=['caseGeneAltered'])
    return dict(muts['Tumor_Sample_Barcode'].value_counts())


In [6]:
perCaseOncogenicInfo = get_per_case_oncogenic_mut_info(allImpactMuts)
perCaseUniqueOncogenicInfo = get_per_case_unique_gene_oncogenic_mut_info(allImpactMuts)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
perCaseNmutInfo = get_per_case_mut_info()
perCaseUniqueMutInfo = get_per_case_unique_gene_mut_info(allImpactMuts)

In [15]:
listOfDicts = []
for case, nmut in perCaseNmutInfo.items():
    if nmut != 0:
        nOnc = 0
        nGeneOnc = 0
        if case in perCaseOncogenicInfo: nOnc = perCaseOncogenicInfo[case]
        if case in perCaseUniqueOncogenicInfo: nGeneOnc = perCaseUniqueOncogenicInfo[case]
        listOfDicts.append({
            'fracOnc': 1.0*nOnc/nmut,
            'Nmut': nmut,
            'Tumor_Sample_Barcode': case,
            'RatioGenesOncMutToGenesMut': 1.0*nGeneOnc/perCaseUniqueMutInfo[case]
        })
df = pd.DataFrame(listOfDicts)
        

In [17]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/observedOncFraction.tsv', sep='\t', index=False)

**Fraction of Oncogenic Mutations at related genes by mutation burden and signature**

In [8]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')

In [11]:
cancerTypeDict = analysis_utils.get_cancer_type_information(cancerTypeDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/mskImpactAsOfMarch2019/dmp/mskimpact/data_clinical_sample.txt', mode='pid')
impactSigs['pid'] = impactSigs['Tumor_Sample_Barcode'].apply(lambda x: x[:9])
impactSigs['cancer_type'] = impactSigs['pid'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)


In [24]:
allImpactMuts = maf_analysis_utils.fix_mll_genes(allImpactMuts)

lets get dfs of all the normal mutations

In [16]:
normalThreshContinousCancers = 30
hyperThreshContinousCancers = 50

normalLung = set(impactSigs[(impactSigs['Nmut_Mb'] < normalThreshContinousCancers) & (impactSigs['cancer_type'] == 'Non-Small Cell Lung Cancer')]['Tumor_Sample_Barcode'])
normalBladder = set(impactSigs[(impactSigs['Nmut_Mb'] < normalThreshContinousCancers) & (impactSigs['cancer_type'] == 'Bladder Cancer')]['Tumor_Sample_Barcode'])
normalMelanoma = set(impactSigs[(impactSigs['Nmut_Mb'] < normalThreshContinousCancers) & (impactSigs['cancer_type'] == 'Melanoma')]['Tumor_Sample_Barcode'])

hyperLung = set(impactSigs[(impactSigs['Nmut_Mb'] > hyperThreshContinousCancers) & (impactSigs['cancer_type'] == 'Non-Small Cell Lung Cancer')]['Tumor_Sample_Barcode'])
hyperBladder = set(impactSigs[(impactSigs['Nmut_Mb'] > hyperThreshContinousCancers) & (impactSigs['cancer_type'] == 'Bladder Cancer')]['Tumor_Sample_Barcode'])
hyperMelanoma = set(impactSigs[(impactSigs['Nmut_Mb'] > hyperThreshContinousCancers) & (impactSigs['cancer_type'] == 'Melanoma')]['Tumor_Sample_Barcode'])


In [18]:
normalEndometrial = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Normal')
normalColorectal = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Normal')
normalGlioma = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Normal')

hyperEndometrial = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
hyperColorectal = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
hyperGlioma = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')


In [25]:
lungNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalLung)]
bladderNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalBladder)]
melanomaNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalMelanoma)]

endometrialNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalEndometrial)]
colorectalNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalColorectal)]
gliomaNormalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(normalGlioma)]

In [ ]:
lungHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperLung)]
lungHyperDf['cancer_type'] = 'Non-Small Cell Lung Cancer'
bladderHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperBladder)]
bladderHyperDf['cancer_type'] = 'Bladder Cancer'
melanomaHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperMelanoma)]
melanomaHyperDf['cancer_type'] = 'Melanoma'

endometrialHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperEndometrial)]
endometrialHyperDf['cancer_type'] = 'Endometrial Cancer'
colorectalHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperColorectal)]
colorectalHyperDf['cancer_type'] = 'Colorectal Cancer'
gliomaHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(hyperGlioma)]
gliomaHyperDf['cancer_type'] = 'Glioma'

In [38]:
allNormalDf = pd.concat([lungNormalDf, bladderNormalDf, melanomaNormalDf, endometrialNormalDf, colorectalNormalDf, gliomaNormalDf])
allHypermutantMuts = pd.concat([lungHyperDf, bladderHyperDf, melanomaHyperDf, endometrialHyperDf, colorectalHyperDf, gliomaHyperDf])


In [28]:
relatedGenesDict = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allNormalDf, pathPrefix,
                                                                                    cancerTypes=['Non-Small Cell Lung Cancer', 'Bladder Cancer', 'Melanoma',
                                                                                                'Endometrial Cancer', 'Colorectal Cancer', 'Glioma'],
                                                                                    t=0.03)



Non-Small Cell Lung Cancer
Bladder Cancer
Melanoma
Endometrial Cancer
Colorectal Cancer
Glioma


In [55]:
def summarize_related_gene_mut_fraction_for_hypermutators(hypermutatorMaf, relatedGenesDict, nmutMbCutoff = 35):
    listOfDicts = []
    for case in set(hypermutatorMaf['Tumor_Sample_Barcode']):
        caseMaf = hypermutatorMaf[hypermutatorMaf['Tumor_Sample_Barcode'] == case]
        caseMafOnc = caseMaf[caseMaf['oncogenic'].notnull()]
        
        cancerType = caseMaf['cancer_type'].iloc[0]
        nmut = caseMaf.shape[0]
        nmut_mb = maf_analysis_utils.get_nmut_mb_from_impact_id(case, nmut)
        if nmut_mb < nmutMbCutoff:
            pass #ignore cases where the nmut_mb is less than 30
        else:
            nOnc = caseMafOnc.shape[0]
            if nOnc > 0:
                nOncRelated = caseMafOnc[caseMafOnc['Hugo_Symbol'].isin(relatedGenesDict[cancerType])].shape[0]
                relatedFrac = (1.0*nOncRelated)/nOnc
                listOfDicts.append({
                    'Tumor_Sample_Barcode': case, 'nOnc': nOnc, 'nOncRelated': nOncRelated,
                    'relatedFrac': relatedFrac, 'cancer_type': cancerType, 'Nmut_Mb': nmut_mb
                })
    return pd.DataFrame(listOfDicts)

In [56]:
df = summarize_related_gene_mut_fraction_for_hypermutators(allHypermutantMuts, relatedGenesDict)

In [57]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/relatedGeneFracByCancerType.tsv', index=False, sep='\t')